In [ ]:
# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Enable inline plotting at lower left
%matplotlib inline

In [ ]:
import os

# Progress bar
from tqdm.auto import trange, tqdm

from astropy.io import fits
from webbpsf_ext import robust

In [ ]:
# Import the JWST class from spaceKLIP

from spaceKLIP.engine import JWST

In [ ]:
# Provide config file, or 'template' to use existing template
config_file = '/home/jarronl/GitHub/spaceKLIP/tests/HIP65426_NIRCam_JML.yaml'
pipeline = JWST(config_file)

In [ ]:
# If desired, sort uncals files from MAST $JWSTDOWNLOAD_OUTDIR to SpaceKLIP directorys
# Creates symbolic links
pipeline.sort_files()

In [ ]:
# Perform ramp fitting
pipeline.rampfit()

In [ ]:
# Perform image processing
pipeline.imgprocess()

In [ ]:
# Compare with no outlier correction
pipeline.meta.outlier_corr = 'none'
pipeline.imgprocess()

In [ ]:
p1 = '/data/NIRData/MAST_proc/01386/HIP65426/F300M_MASK335R/IMGPROCESS/SCI+REF/'
p2 = '/data/NIRData/MAST_proc/01386/HIP65426/F300M_MASK335R/IMGPROCESS/SCI+REF_CLEAN/'
f = 'jw01386003001_03107_00001_nrcalong_calints.fits'
hdul1 = fits.open(p1+f)
hdul2 = fits.open(p2+f)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(14,7))

im = hdul1[1].data[0]

med = np.nanmedian(im)
std = robust.medabsdev(im)
vmin = med - 5*std
vmax = med + 5*std

ax = axes[0]
ax.imshow(im, vmin=vmin, vmax=vmax)


im = hdul2[1].data[0]

med = np.nanmedian(im)
std = robust.medabsdev(im)
vmin = med - 5*std
vmax = med + 5*std

ax = axes[1]
ax.imshow(im, vmin=vmin, vmax=vmax)


fig.tight_layout()

In [ ]:
# Perform PSF Subtraction
pipeline.subtraction()

In [ ]:
# Compute raw contrast
pipeline.raw_contrast()

In [ ]:
# Compute calibrated contrast
pipeline.cal_contrast()

In [ ]:
# Extract companion photometry and astrometry
pipeline.companion()